# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd

In [2]:
# load data from database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql('SELECT * FROM InsertTableName', con=engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#Define feature and target variables X and Y
import numpy as np
X = df['message']
y = df.iloc[:, 4:]
y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [4]:
import nltk
import re
nltk.download(['punkt', 'stopwords','wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
def tokenize(text):
    text = re.sub(r'[^\w\s]','',text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tok = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tok

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [5]:
for message in X[:5]:
    print(message)
    print(tokenize(message))

Weather update - a cold front from Cuba that could pass over Haiti
['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over
['hurricane']
Looking for someone but no name
['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.
['un', 'report', 'leogane', '8090', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight
['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)), 
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

y_pred = pipeline.predict(X_test)

y_pred_df = pd.DataFrame(y_pred, columns = y_test.columns)

evaluation = {}

for column in y_test.columns:
    evaluation[column] = []
    evaluation[column].append(precision_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(recall_score(y_test[column], y_pred_df[column]))
    evaluation[column].append(f1_score(y_test[column], y_pred_df[column]))

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
pd.DataFrame(evaluation)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,0.843532,0.782497,0.0,0.729403,0.510417,0.780000,0.736842,0.0,0.371429,0.0,...,0.0,0.0,0.827723,0.804020,0.747596,0.0,0.876289,0.869565,0.611111,0.729560
1,0.908712,0.456456,0.0,0.599565,0.076803,0.093301,0.072917,0.0,0.053061,0.0,...,0.0,0.0,0.579750,0.256822,0.421409,0.0,0.726496,0.116959,0.053012,0.300518
2,0.874910,0.576577,0.0,0.658142,0.133515,0.166667,0.132701,0.0,0.092857,0.0,...,0.0,0.0,0.681892,0.389294,0.538995,0.0,0.794393,0.206186,0.097561,0.425688


In [10]:
# Classification report
for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Column: related
             precision    recall  f1-score   support

          0       0.61      0.46      0.53      1873
          1       0.84      0.91      0.87      5992

avg / total       0.79      0.80      0.79      7865

Column: request
             precision    recall  f1-score   support

          0       0.90      0.97      0.93      6533
          1       0.78      0.46      0.58      1332

avg / total       0.88      0.89      0.87      7865

Column: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7829
          1       0.00      0.00      0.00        36

avg / total       0.99      1.00      0.99      7865

Column: aid_related
             precision    recall  f1-score   support

          0       0.75      0.85      0.80      4646
          1       0.73      0.60      0.66      3219

avg / total       0.74      0.75      0.74      7865

Column: medical_help
             precision    recall  f1-score   support


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f3e2fd16b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [12]:
from sklearn.model_selection import GridSearchCV

parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'vect__max_df': [0.75, 1.0],
              'tfidf__use_idf': [True, False]}

cv = GridSearchCV(pipeline, param_grid = parameters, verbose = 3, cv = 2)

model = cv.fit(X_train, y_train)
y_pred = model.predict(X_test)

Fitting 2 folds for each of 8 candidates, totalling 16 fits
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 
[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.23637750653879686, total= 1.7min
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.4min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 1), score=0.23455040871934604, total= 1.7min
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.7min remaining:    0.0s


[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.2382301656495205, total= 2.0min
[CV] tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2) 
[CV]  tfidf__use_idf=True, vect__max_df=0.75, vect__ngram_range=(1, 2), score=0.23553133514986377, total= 2.1min
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 1) .
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.236050566695728, total= 1.7min
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 1) .
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 1), score=0.22713896457765667, total= 1.7min
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 2) .
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.24182650392327812, total= 2.0min
[CV] tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 2) .
[CV]  tfidf__use_idf=True, vect__max_df=1.0, vect__ngram_range=(1, 2), score=0.2288828

[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed: 41.1min finished


In [13]:
print('Best Parameters: {}'.format(model.best_params_))

Best Parameters: {'tfidf__use_idf': False, 'vect__max_df': 0.75, 'vect__ngram_range': (1, 2)}


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
print('Accuracy:')
print((y_pred == y_test).mean())
for i in range(35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.795168
request                   0.881755
offer                     0.995423
aid_related               0.737826
medical_help              0.921678
medical_products          0.950413
search_and_rescue         0.977368
security                  0.981310
military                  0.968849
child_alone               1.000000
water                     0.952193
food                      0.930070
shelter                   0.923077
clothing                  0.985760
money                     0.979148
missing_people            0.988303
refugees                  0.967196
death                     0.958551
other_aid                 0.871583
infrastructure_related    0.935537
transport                 0.955753
buildings                 0.955245
electricity               0.979530
tools                     0.994024
hospitals                 0.990083
shops                     0.996440
aid_centers               0.986777
other_infrastructure      0.957025
weather_re

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
from sklearn.ensemble import AdaBoostClassifier

pipeline = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', MultiOutputClassifier(AdaBoostClassifier()))])

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print('Accuracy:')
print((y_pred == y_test).mean())
for i in range (35):
    print("Column: {}".format(y_test.columns[i]))
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy:
related                   0.792244
request                   0.898156
offer                     0.993897
aid_related               0.747362
medical_help              0.924221
medical_products          0.954990
search_and_rescue         0.975969
security                  0.979402
military                  0.970121
child_alone               1.000000
water                     0.960712
food                      0.947997
shelter                   0.941640
clothing                  0.987667
money                     0.979275
missing_people            0.989320
refugees                  0.968341
death                     0.966815
other_aid                 0.870947
infrastructure_related    0.933884
transport                 0.958678
buildings                 0.961348
electricity               0.980165
tools                     0.991990
hospitals                 0.987031
shops                     0.996058
aid_centers               0.985251
other_infrastructure      0.951939
weather_re

### 9. Export your model as a pickle file

In [18]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.